In [1]:
from datetime import datetime, timedelta
import os

from ipyfilechooser import FileChooser
import ipywidgets as widgets
import pandas as pd
import geemap
import ee

from setting.utils import SelectFilesButton, monthlist, date_format_concersion, read_shp_date_data, read_bands_statics, make_temp_file, cbind_era5
from setting.config import settings

##### 先跟Google Earth Engine取得init以及雲端專案連線

In [2]:
# After executing this line of code for the first use, you can get the authentication number linked to Google.
Map = geemap.Map()
# Authenticate the Google earth engine with google account
ee.Initialize() 

先上傳要用的shapfile檔案  ```註:大小不可超過2.5MB```  

In [3]:
my_button = SelectFilesButton()
my_button


SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

Regional category 這個欄位會讀取輸入的shp檔裡面所有欄位  
選取要分析的欄位表  
輸入起始日期跟結束日期
```註:每個衛星程式的支援時間點都不太一樣```  
這一支的網址是:[ERA5-Land Daily Aggregated - ECMWF Climate Reanalysis](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_DAILY_AGGR)

In [4]:
file_name, start, end = read_shp_date_data(my_button)
widgets.HBox([file_name, start, end])

選取需要得到的Band跟Zonal要做的統計值  
```註:統計值一次只能選一個，但Band可以一次多個```

In [7]:
band_name, statics = read_bands_statics(settings.era5_bands_list)
widgets.HBox([band_name, statics])

建立temp table暫存fodler跟選擇要輸出檔案位置跟檔案名稱  
```註:如果是用mac系統且vscode平台的話這個功能目前會無法使用，建議可以用Jupyter notebook開啟```

In [8]:
# give the output floder and flie name
folder_name = make_temp_file('data_all_google_earth_engine_era5')
folder = FileChooser()
display(folder)

FileChooser(path='D:\Projects\TMU\gee_dengue\zonal statistic', filename='', title='', show_hidden=False, selec…

In [11]:
time_list = monthlist(start.value, end.value)
states = geemap.shp_to_ee("".join(my_button.files))

# Filter to specific polygons by ID_2
selected_ids = [89, 98, 160, 171, 434, 444]  # Replace with your desired IDs
filtered_states = states.filter(ee.Filter.inList('ID_2', selected_ids))

for i in range(0, len(time_list)):
    Era5 = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
            .filter(ee.Filter.date(
                datetime.strptime(time_list[i][0], "%Y-%m-%d"),
                datetime.strptime(time_list[i][1], "%Y-%m-%d") + timedelta(days=1)
            )) \
            .map(lambda image: image.select(band_name.value)) \
            .map(lambda image: image.clip(filtered_states)) \
            .map(lambda image: image.reproject(crs=settings.crs))

    Era5 = Era5.toBands()
    
    out_dir = os.path.expanduser(folder_name)
    out_dem_stats = os.path.join(out_dir, 'Era5_{}_{}.csv'.format(statics.value, time_list[i]))

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    geemap.zonal_statistics(
        Era5, filtered_states, out_dem_stats,
        statistics_type=statics.value, scale=1000
    )

    data_temp = pd.read_csv(out_dem_stats)
    data = []

    if len(band_name.value) == 1 and time_list[i][0] == time_list[i][1]:
        df = data_temp.copy()
        df[file_name.value] = data_temp.loc[:, [file_name.value]]
        df['Date'] = date_format_concersion(time_list[i][0], output_format='%Y/%m/%d')
        df['Doy'] = datetime.strptime(time_list[i][0], '%Y-%m-%d').strftime('%j')
        select_columns = ['Date', 'Doy'] + [item.lower() for item in band_name.value] + [file_name.value]
        df = df[select_columns]
        df.columns = ['Date', 'Doy'] + band_name.value + [file_name.value]
        data.append(df)

    else:
        column_name_list = data_temp.columns.tolist()
        data = []

        for j in range(0, len(column_name_list), len(band_name.value)):
            date_str = data_temp.columns[j][:8]

            if date_str.isdigit():
                df = data_temp.loc[:, column_name_list[j:j + len(band_name.value)]]
                df[file_name.value] = data_temp.loc[:, [file_name.value]]

                df.insert(0, 'Date', date_format_concersion(date_str, output_format='%Y/%m/%d'))
                df.insert(1, 'Doy', datetime.strptime(date_str, '%Y%m%d').strftime('%j'))

                colnames = ['Date', 'Doy'] + list(band_name.value) + [file_name.value]

                df.columns = colnames

                data.append(df)

    appended_data = pd.concat(data, axis=0, ignore_index=True)
    appended_data.to_csv(out_dem_stats, index=False)


Computing statistics ...


Generating URL ...
Please wait ...
Data downloaded to d:\Projects\TMU\gee_dengue\zonal statistic\data_all_google_earth_engine_era5\Era5_MEAN_['2013-01-01', '2013-01-31'].csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\Projects\TMU\gee_dengue\zonal statistic\data_all_google_earth_engine_era5\Era5_MEAN_['2013-02-01', '2013-02-28'].csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\Projects\TMU\gee_dengue\zonal statistic\data_all_google_earth_engine_era5\Era5_MEAN_['2013-03-01', '2013-03-31'].csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\Projects\TMU\gee_dengue\zonal statistic\data_all_google_earth_engine_era5\Era5_MEAN_['2013-04-01', '2013-04-30'].csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\Projects\TMU\gee_dengue\zonal statistic\data_all_google_earth_engine_era5\Era5_MEAN_['2013-05-01', '2013-05-31'].csv
Computing statistics ...
Gener

Generating URL ...
Please wait ...
Data downloaded to d:\Projects\TMU\gee_dengue\zonal statistic\data_all_google_earth_engine_era5\Era5_MEAN_['2014-08-01', '2014-08-31'].csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\Projects\TMU\gee_dengue\zonal statistic\data_all_google_earth_engine_era5\Era5_MEAN_['2014-09-01', '2014-09-30'].csv
Computing statistics ...
Generating URL ...


Please wait ...
Data downloaded to d:\Projects\TMU\gee_dengue\zonal statistic\data_all_google_earth_engine_era5\Era5_MEAN_['2014-10-01', '2014-10-31'].csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\Projects\TMU\gee_dengue\zonal statistic\data_all_google_earth_engine_era5\Era5_MEAN_['2014-11-01', '2014-11-30'].csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\Projects\TMU\gee_dengue\zonal statistic\data_all_google_earth_engine_era5\Era5_MEAN_['2014-12-01', '2014-12-31'].csv
Computing statistics ...
Generating URL ...


An error occurred while downloading.


Exception: local variable 'r' referenced before assignment

In [30]:
cbind_era5(statics.value, out_dir, band_name.value, folder)